In [1]:
import os
os.chdir("../../")#与easymlops同级目录

### 拆分数据

In [2]:
import pandas as pd
data=pd.read_csv("./data/demo.csv")
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
x_train=data[:500]
x_test=data[500:]
y_train=x_train["Survived"]
y_test=x_test["Survived"]
del x_train["Survived"]
del x_test["Survived"]

### 切片式
可以通过切分的方式获取部分连续的pipe模块，并组装为Pipeline，这样可以更加方便灵活的获取中间结果

In [4]:
from easymlops import TablePipeLine
from easymlops.table.preprocessing import *
from easymlops.table.encoding import *
from easymlops.table.perfopt import *
from easymlops.table.decomposition import *

In [5]:
table=TablePipeLine()
table.pipe(FixInput())\
  .pipe(FillNa())\
  .pipe(OneHotEncoding(cols=["Pclass","Sex"],drop_col=False))\
  .pipe(WOEEncoding(cols=["Sex","Pclass"],y=y_train))\
  .pipe(LabelEncoding(cols=["Name","Ticket"]))\
  .pipe(TargetEncoding(cols=["Embarked","Cabin"],y=y_train,name="target_encoding"))\
  .pipe(FillNa())\
  .pipe(Normalizer())\
  .pipe(PCADecomposition(n_components=8))

table.fit(x_train)

In [6]:
#运行前3层
step1=table[:3].transform(x_test[:5])
step1

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female
500,501,3,"Calic, Mr. Petar",male,17.0,0,0,315086,8.664062,nan,S,1,0,0,1,0
501,502,3,"Canavan, Miss. Mary",female,21.0,0,0,364846,7.750000,nan,Q,1,0,0,0,1
502,503,3,"O'Sullivan, Miss. Bridget Mary",female,0.0,0,0,330909,7.628906,nan,Q,1,0,0,0,1
503,504,3,"Laitinen, Miss. Kristina Sofia",female,37.0,0,0,4135,9.585938,nan,S,1,0,0,0,1
504,505,1,"Maioni, Miss. Roberta",female,16.0,0,0,110152,86.500000,B79,S,0,1,0,0,1


In [7]:
#运行中间3,4层
step2=table[3:5].transform(step1)
step2

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Pclass_3,Pclass_1,Pclass_2,Sex_male,Sex_female
500,501,0.482439,0,1.111379,17.0,0,0,0,8.664062,nan,S,1,0,0,1,0
501,502,0.482439,0,-1.569990,21.0,0,0,0,7.750000,nan,Q,1,0,0,0,1
502,503,0.482439,0,-1.569990,0.0,0,0,0,7.628906,nan,Q,1,0,0,0,1
503,504,0.482439,0,-1.569990,37.0,0,0,0,9.585938,nan,S,1,0,0,0,1
504,505,-0.741789,0,-1.569990,16.0,0,0,231,86.500000,B79,S,0,1,0,0,1


In [8]:
#运行5层及之后
step3=table[5:].transform(step2)
step3

,0,1,2,3,4,5,6,7
500,-2.148420,0.307872,0.526932,0.088112,0.324999,-0.238535,-0.432583,-0.424289
501,0.212949,2.322017,-1.286037,-0.469809,1.537922,-0.098157,0.775934,0.459946
502,0.053812,2.543002,-1.504793,-0.263976,1.525352,0.729823,0.737169,-0.240717
503,0.089901,2.005645,-1.130925,-0.942879,0.544493,-1.649309,-0.119709,-0.125324
504,2.673423,0.110068,-0.091175,-0.154845,0.265834,-0.959033,1.617030,-2.494035
